In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn


import sklearn
from sklearn import preprocessing
from datetime import datetime
import random
import math
import holidays
us_holidays = holidays.UnitedStates()


In [2]:
# train_df = pd.read_feather('Data/train.feather')
# weather_train_df = pd.read_feather('Data/weather_train_processed_lag.feather')
test_df = pd.read_feather('Data/test.feather')
weather_test_df = pd.read_feather('Data/weather_test_processed_lag.feather')
building_meta_df = pd.read_feather('Data/building_metadata.feather')

In [3]:
%%time
test_df = test_df.merge(building_meta_df, left_on = "building_id", right_on = "building_id", how = "left")
test_df = test_df.merge(weather_test_df, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")

Wall time: 35.8 s


In [4]:
train_df = test_df
# del train_df

In [6]:
%%time
le = preprocessing.LabelEncoder()
train_df['primary_use'] = le.fit_transform(train_df['primary_use'])

move = ['timestamp'] + [x for x in train_df.columns if x not in ['meter_reading', 'timestamp']]
train_df = train_df[move]

mask =(train_df['site_id']==0) & (train_df['timestamp'] < '2016-06-01')
train_df = train_df[~mask]

# this may take mins
train_df['date_month'] = train_df['timestamp'].map(lambda x: x.month)
train_df['date_day'] = train_df['timestamp'].map(lambda x: x.day)
train_df['iso_week'] = train_df['timestamp'].map(lambda x: x.isocalendar()[1])
train_df['iso_weekend'] = train_df['timestamp'].map(lambda x: x.isocalendar()[2])
train_df['hour'] = train_df['timestamp'].map(lambda x: x.hour)
train_df['holidays'] = train_df['timestamp'].map(lambda x: 1 if x in us_holidays else 0)

Wall time: 10min 3s


In [8]:
from sklearn import preprocessing

train_df['wind_direction cat'][train_df['wind_direction cat'].isna()] = 'Unknown'
train_df['wind_direction cat2'][train_df['wind_direction cat2'].isna()] = 'Unknown'
train_df['wind_direction cat_lag'][train_df['wind_direction cat_lag'].isna()] = 'Unknown'
train_df['wind_direction cat2_lag'][train_df['wind_direction cat2_lag'].isna()] = 'Unknown'

cat_need_encode = ['wind_direction cat', 'wind_direction cat2',
                  'wind_direction cat_lag', 'wind_direction cat2_lag', 
                  ]

for var in cat_need_encode:
    print(var)
    le = preprocessing.LabelEncoder()
    train_df[var] = le.fit_transform(train_df[var])

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

wind_direction cat
wind_direction cat2
wind_direction cat_lag
wind_direction cat2_lag


In [10]:
use = [ 'building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'wind_direction cat', 'wind_direction cat2',
        'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction cat_lag', 'wind_direction cat2_lag',
        'date_month', 'date_day',
       'iso_week', 'iso_weekend', 'hour', 'holidays']

In [11]:
train_df = train_df[use]

In [12]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [13]:
train_df = reduce_mem_usage(train_df)

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

Mem. usage decreased to 2346.19 Mb (75.4% reduction)


In [15]:
use = ['building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'wind_direction_cat', 'wind_direction_cat2',
        'air_temperature_lag',
       'cloud_coverage_lag', 'dew_temperature_lag', 'precip_depth_1_hr_lag',
       'sea_level_pressure_lag', 'wind_direction_lag', 'wind_speed_lag',
       'wind_direction_cat_lag', 'wind_direction_cat2_lag',
        'date_month', 'date_day',
       'iso_week', 'iso_weekend', 'hour', 'holidays']

train_df.columns = use

In [16]:
%%time
train_df = train_df.reset_index(drop=True)
train_df.to_csv('processed_automl_google_test.csv', index=False)

Wall time: 14min 12s


In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 31 columns):
building_id                int16
meter                      int8
site_id                    int8
primary_use                int8
square_feet                int32
year_built                 float16
floor_count                float16
air_temperature            float16
cloud_coverage             float16
dew_temperature            float16
precip_depth_1_hr          float16
sea_level_pressure         float16
wind_direction             float16
wind_speed                 float16
wind_direction_cat         int8
wind_direction_cat2        int8
air_temperature_lag        float16
cloud_coverage_lag         float16
dew_temperature_lag        float16
precip_depth_1_hr_lag      float16
sea_level_pressure_lag     float16
wind_direction_lag         float16
wind_speed_lag             float16
wind_direction_cat_lag     int8
wind_direction_cat2_lag    int8
date_month                 int8
dat

## check the order

In [8]:
df = pd.read_csv('processed_automl_google_test.csv')

In [3]:
df.shape

(41697600, 31)

In [40]:
use = ['building_id', 'meter', 
        'date_month', 'date_day',
       'hour', 'air_temperature', 'wind_direction', 'air_temperature_lag', 'wind_direction_lag',
      'dew_temperature', 'dew_temperature_lag',
      'sea_level_pressure', 'sea_level_pressure_lag',
      'precip_depth_1_hr', 'precip_depth_1_hr_lag', 'precip_depth_1_hr',
      'cloud_coverage']

df2 = df[use]

# del df

In [41]:
%%time
df3 = df2.drop_duplicates()

Wall time: 1min 18s


In [43]:
df3.shape

(41696189, 17)

In [44]:
df3.shape[0]/df.shape[0]

0.999966161121983